# Multi Author Writing Style Analysis
by: Noah Syrkis


As a writer, I have always been interested in the idea of a "writing style".
I have always wondered if there was a way to quantify the style of a writer, and if so, how would one go about doing so?
This project is an attempt to build a model that does precisely that.

In [1]:
from functools import partial
from src.utils import get_data

## Data

In [2]:
data = get_data(1)

 77%|███████▋  | 690/900 [03:11<00:58,  3.60it/s]


KeyboardInterrupt: 